# Lab 6

Scikit learn provides a large variety of algorithms for some common Machine Learning tasks, such as:

* Classification
* Regression
* Clustering
* Feature Selection
* Anomaly Detection

It also provides some datasets that you can use to test these algorithms:

* Classification Datasets:
    * Breast cancer wisconsin
    * Iris plants (3-classes)
    * Optical recognition of handwritten digits (10-classes)
    * Wine (n-classes)

* Regression Datasets: 
    * Boston house prices 
    * Diabetes
    * Linnerrud (multiple regression)
    * California Housing

* Image: 
    * The Olivetti faces
    * The Labeled Faces in the Wild face recognition
    * Forest covertypes

* NLP:
    * News group
    * Reuters Corpus Volume I 

* Other:
    * Kddcup 99- Intrusion Detection

## Exercises

1. Use the full [Kddcup](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) dataset to compare classification performance of 3 different classifiers. 
    * Separate the data into train, validation, and test. 
    * Use accuracy as the metric for assessing performance. 
    * For each classifier, identify the hyperparameters. Perform optimization over at least 2 hyperparameters.   
    * Compare the performance of the optimal configuration of the classifiers.

2. Pick the best algorithm in question 1. Create an ensemble of at least 25 models, and use them for the classification task. Identify the top and bottom 10% of the data in terms of uncertainty of the decision.

3. Use 2 different feature selection algorithm to identify the 10 most important features for the task in question 1. Retrain classifiers in question 1 with just this subset of features and compare performance.

4. Use the same data, removing the labels, and compare performance of 3 different clustering algorithms. Can you find clusters for each of the classes in question 1? 

5. Can you identify any clusters within the top/botton 10% identified in 2. What are their characteristics?

6. Use the "SA" dataset to compare the performance of 3 different anomaly detection algorithms.

7. Create a subsample of 250 datapoints, redo question 6, using Leave-one-out as the method of evaluation.

8. Use the feature selection algorithm to identify the 5 most important features for the task in question 6, for each algorithm. Does the anomaly detection improve using less features?

## Quick look at the data

In [1]:
from sklearn.datasets import fetch_kddcup99
D=fetch_kddcup99()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dir(D)

['DESCR', 'data', 'feature_names', 'frame', 'target', 'target_names']

In [32]:
type(D)

sklearn.utils._bunch.Bunch

In [3]:
print(D["DESCR"])

.. _kddcup99_dataset:

Kddcup 99 dataset
-----------------

The KDD Cup '99 dataset was created by processing the tcpdump portions
of the 1998 DARPA Intrusion Detection System (IDS) Evaluation dataset,
created by MIT Lincoln Lab [2]_. The artificial data (described on the `dataset's
homepage <https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html>`_) was
generated using a closed network and hand-injected attacks to produce a
large number of different types of attack with normal activity in the
background. As the initial goal was to produce a large training set for
supervised learning algorithms, there is a large proportion (80.1%) of
abnormal data which is unrealistic in real world, and inappropriate for
unsupervised anomaly detection which aims at detecting 'abnormal' data, i.e.:

* qualitatively different from normal data
* in large minority among the observations.

We thus transform the KDD Data set into two different data sets: SA and SF.

* SA is obtained by simply selecting all

In [5]:
import numpy as np
np.unique(D["target"])

array([b'back.', b'buffer_overflow.', b'ftp_write.', b'guess_passwd.',
       b'imap.', b'ipsweep.', b'land.', b'loadmodule.', b'multihop.',
       b'neptune.', b'nmap.', b'normal.', b'perl.', b'phf.', b'pod.',
       b'portsweep.', b'rootkit.', b'satan.', b'smurf.', b'spy.',
       b'teardrop.', b'warezclient.', b'warezmaster.'], dtype=object)

In [6]:
len(np.unique(D["target"]))

23

In [7]:
D["feature_names"]

['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']

## Part 1

### a) Separate the data into train, validation, and test.

In [40]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer


'''
Use the full Kddcup dataset to compare classification performance of 3 different classifiers.

Separate the data into train, validation, and test.
Use accuracy as the metric for assessing performance.
For each classifier, identify the hyperparameters. Perform optimization over at least 2 hyperparameters.
Compare the performance of the optimal configuration of the classifiers.
'''


'\nUse the full Kddcup dataset to compare classification performance of 3 different classifiers.\n\nSeparate the data into train, validation, and test.\nUse accuracy as the metric for assessing performance.\nFor each classifier, identify the hyperparameters. Perform optimization over at least 2 hyperparameters.\nCompare the performance of the optimal configuration of the classifiers.\n'

In [41]:
X = D.data
y = D.target

'''
To avoid this error:

ValueError: You appear to be using a legacy multi-label data representation. 
Sequence of sequences are no longer supported; use a binary array or sparse matrix 
instead - the MultiLabelBinarizer transformer can convert to this format.

y_binary is implemented rather than y directly
'''
label_encoder = LabelEncoder()
y_binary = label_encoder.fit_transform(y)

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2)

# Further split the previously made training data into training (80%) and validation (20%) sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# Print the shapes of the resulting sets
print("X_train & y_train shapes: X={}, y={}".format(X_train.shape, y_train.shape))
print("X_val & y_val shapes:     X={}, y={}".format(X_val.shape, y_val.shape))
print("X_test & y_test shapes:   X={}, y={}".format(X_test.shape, y_test.shape))

X_train & y_train shapes: X=(316172, 41), y=(316172,)
X_val & y_val shapes:     X=(79044, 41), y=(79044,)
X_test & y_test shapes:   X=(98805, 41), y=(98805,)


### b) Set up classifiers using accuracy as the metric for assessing performance.
Classifiers are AdaBoost, RandomForest, and GaussianProcess.

In [45]:

# Define classifiers
adaboost_classifier = AdaBoostClassifier()
random_forest_classifier = RandomForestClassifier()
gaussian_process_classifier = GaussianProcessClassifier()


In [46]:
# Get hyper parameters

# AdaBoost hyperperameters
adaboost_params = adaboost_classifier.get_params()
print(adaboost_params)

# Random Forest hyperparameters
rf_params = random_forest_classifier.get_params()
print(rf_params)

# Gaussian Process hyperparameters
gauss_params = gaussian_process_classifier.get_params()
print(gauss_params)

{'algorithm': 'SAMME.R', 'base_estimator': 'deprecated', 'estimator': None, 'learning_rate': 1.0, 'n_estimators': 50, 'random_state': None}
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'copy_X_train': True, 'kernel': None, 'max_iter_predict': 100, 'multi_class': 'one_vs_rest', 'n_jobs': None, 'n_restarts_optimizer': 0, 'optimizer': 'fmin_l_bfgs_b', 'random_state': None, 'warm_start': False}


In [55]:
# for adaboost optimize learning_rate, and n_estimators
# for RandForest optimize n_estimators, and max_depth
# for GaussProc optimize max_iter_predict, and n_jobs

# Define the hyperparameter grid
adaboost_param_grid = {'n_estimators': [50, 100, 200],
                       'learning_rate': [0.01, 0.1, 1.0]
                      }
rf_param_grid = {'n_estimators':[50, 100, 200], 
                 'max_depth': [None, 10, 20, 30]
                }
gauss_param_grid = {'max_iter_predict': [50, 100, 200], 
                    'n_jobs':[1, 3, 7]
                   }

# Define GridSearchCV object for each classifier. Scoring metric for all is accuracy.
adaboost_grid_search = GridSearchCV(adaboost_classifier,
                                   adaboost_param_grid,
                                   scoring='accuracy',
                                   cv=2,
                                   error_score='raise')
rf_grid_search = GridSearchCV(random_forest_classifier,
                             rf_param_grid,
                             scoring='accuracy',
                             cv=2,
                             error_score='raise')
gauss_grid_search = GridSearchCV(gaussian_process_classifier, 
                                 gauss_param_grid, 
                                 scoring='accuracy', 
                                 cv=2,
                                error_score='raise')

In [58]:
# Fit GridSearchCV object on training data for each classifier
adaboost_grid_search.fit(X_train, y_train)
rf_grid_search.fit(X_train, y_train)
gauss_grid_search.fit(X_train, y_train)


/home/reese_quinto/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


ValueError: could not convert string to float: b'tcp'